In [1]:
import pandas as pd
import os
import re
import csv
import time
import requests

# 合并数据

In [2]:
for file in os.listdir(r".\result"):
    with open(r".\result\\"+file, "r", encoding='utf-8') as f1:
        content = f1.read()
        save_file = re.search(u"[\u4e00-\u9fa5]+", file)[0]
        with open(r".\\%s.csv"%save_file, 'a',encoding='utf-8') as f2:
            f2.write(content)
print("合并成功")

合并成功


In [3]:
for file in  ['住宅用地.csv', '公共管理与公共服务用地.csv', '商服用地.csv', '工矿仓储用地.csv']:
# file = '工矿仓储用地'
    pd.read_csv(r".\\"+file,names=["区县",'url']).drop_duplicates("url").to_csv(r".\merge_data\\"+file)

# 清洗数据

In [84]:
data = pd.read_csv(r".\result\工矿仓储用地.csv",index_col=0, encoding='gbk')

In [85]:
data = data.rename(columns={'交易时间':'成交时间'})

In [86]:
data["成交年份"] = data["成交时间"].astype(str).str[:4]

In [87]:
def parse_time(date):
    try:
        date = date.replace("年","/")
        date = date.replace("月","/")
        date = date.replace("日","")
        return date
    except:
        print(date)
data["成交时间"] = data["成交时间"].apply(parse_time)

In [88]:
def parse_date(date):
    date = re.sub('/',"年", date,count=1)
    date = re.sub('/',"月", date,count=1)
    new_date = re.search("(\S+月)", date)[0]
    return new_date
data['成交年月'] = data["成交时间"].apply(parse_date)

In [89]:
data["地块面积（平方米）"] = data['面积（公顷）'] * 10000

In [90]:
data["交易方式"] = data["交易方式"].str.replace("出让", "")

In [91]:
data["行政区"] = data["区县"].str.replace('本级',"")
data["用途"] = data["用途"].str.replace('用地',"")

In [92]:
pattern = "(\S{2}区)|(\S{2}县)"
data["县区"] = data["宗地位置"].astype(str).str.extract(pattern, expand=True)[0]

In [93]:
def is_city(city):
    if city in ['中原区','二七区','管城区','金水区','上街区','惠济区','巩义市','荥阳市','新密市','新郑市','登封市','中牟县',]:
        return city
    else:
        return None
data["县区"] = data["县区"].apply(is_city)

In [94]:
data["县区（新）"] = data["县区"].fillna(data["行政区"])

In [96]:
data['约定容积率（下限）'] = data['约定容积率（下限）'].replace(0, None)

In [97]:
data['容积率'] = data['约定容积率（下限）'].fillna(data['约定容积率（上限）'])

In [82]:
data["建筑面积"] = data["容积率"] * data["地块面积（平方米）"]

TypeError: can't multiply sequence by non-int of type 'float'

In [98]:
data.to_excel(r".\display\工业用地.xlsx",encoding="gbk")

# 具体地址请求

In [47]:
data = pd.read_excel('./display/住宅用地.xlsx')

In [49]:
def parseFunc(city,town):
    if city == '郑州市':
        city = resCity(town)
    return city
data['区县（新）'] = data.apply(lambda row: parseFunc(row['区县'], row['宗地位置']), axis=1)

In [50]:
data.to_excel('住宅用地.xlsx')

In [42]:
def resCity(keywords):
    params = {
        'key': 'd3c362e23d62acb3b586bad0f68bd826',
        'keywords': keywords,
        'city': '郑州',
        'types':190301,
    }
    r = requests.get('https://restapi.amap.com/v3/place/text', params)
    adname = ''
    if r.ok:
        dataJs = r.json()
        for pois in dataJs['pois']:
            adname += pois['adname']
        return adname

In [17]:
data.to_excel('住宅用地.xlsx')

# 清洗数据

In [11]:
path = '工业用地'

In [12]:
data = pd.read_excel('.\display\%s.xlsx'%path, encoding='gbk')

In [13]:
data = data.rename(columns = {"行政区":"省市",'地块面积':"地块面积（平方米）","用途":"土地用途"})

In [ ]:
data

In [14]:
data['地块公告号'] = None
data['起始价格（万元）'] = '/'
data['溢价率'] = '/'
data['限高'] = '/'
data["成交价格（万元）"] = data["成交价格（万元）"].fillna('/')

In [15]:
columns = ['土地用途','省市','区县','地块公告号','宗地名称','宗地位置', '出让年限','交易方式','起始价格（万元）','成交价格（万元）','成交年份','成交年份','成交时间','地块面积（平方米）','容积率','建筑面积（平方米）', '限高','受让单位']

In [16]:
data = data[columns]

KeyError: "['建筑面积（平方米）'] not in index"

In [170]:
data.to_excel("./result/%s.xlsx"%path)